In [10]:
import math
import numpy as np
from sympy.physics.wigner import wigner_3j as _sympy_wigner_3j
%run init_function.ipynb

def k_list(l1, l2):
    return [int(k) for k in range(abs(l1-l2),abs(l1+l2)+1) if (k+l1+l2)%2==0]

def ak(l1, k, l2):
    """Compute the Wigner 3-j symbol (l1 k l2; 0 0 0) for integer l values."""
    # closed-form for (j1 j2 j3; 0 0 0):
    # (see standard angular-momentum references)
    w3j_o = _sympy_wigner_3j(l1, k, l2, 0, 0, 0)
    w3j_squared = float(w3j_o)**2
    a_k = (2 * l2 + 1) * w3j_squared
    return a_k

def slater_integral(r : list, u1 : list, u2 : list, k : int):
    w = r[0] - r[1]
    r_safe = np.maximum(r, 1e-30)

    # 被积函数
    product = u1 * u2

    # 向前累积 Y^k(r) = ∫_0^r r'^k u_i u_j dr'
    integrand_Y = (r**k) * product * w
    Y_k = np.cumsum(integrand_Y)

    # 向后累积 Z^k(r) = ∫_r^∞ r'^(-k-1) u_i u_j dr'
    integrand_Z = (r_safe ** (-k - 1)) * product * w
    Z_k = np.cumsum(integrand_Z[::-1])[::-1]  # 反向累积后翻转

    # 组合 R^k(r) = Y^k(r)/r^(k+1) + Z^k(r) * r^k
    R_k = Y_k / (r_safe ** (k + 1)) + Z_k * (r_safe**k)

    return R_k

def exchange_energy(r, u_target : list, qn_target):
    Ku = np.zeros((len(r), len(r)), dtype=float)
    for (u, qn) in zip(u_funcs, qn_set):
        ks = k_list(qn[1], qn_target[1])
        for k in ks:
            R_k = slater_integral(r, u, u_target, k)
            a_k = ak(qn[1], k, qn_target[1])
            g_m = 2 * qn[1] + 1  # 空间简并度
            n_eff = qn[2] / (g_m * 2.0)  # 每个 (m, σ) 的平均占据
            Ku -= n_eff * a_k * np.outer(R_k, u)  # 累加交换贡献
    return Ku
qn_set = select_qn()
r, u_funcs = radial_function_set(qn_set)
exchange_energy(r, u_funcs[0], qn_set[0])


<>:9: SyntaxWarning: invalid escape sequence '\p'
C:\Users\win\AppData\Local\Temp\ipykernel_9640\2534220277.py:9: SyntaxWarning: invalid escape sequence '\p'
  """


array([[0.00000000e+00, 1.74402577e-29, 3.48805155e-29, ...,
        1.73879368e-26, 1.74053771e-26, 1.74228174e-26],
       [0.00000000e+00, 1.74412625e-29, 3.48825250e-29, ...,
        1.73889386e-26, 1.74063799e-26, 1.74238211e-26],
       [0.00000000e+00, 1.74422481e-29, 3.48844963e-29, ...,
        1.73899213e-26, 1.74073635e-26, 1.74248058e-26],
       ...,
       [0.00000000e+00, 1.19350991e-29, 2.38701981e-29, ...,
        1.18992937e-26, 1.19112288e-26, 1.19231639e-26],
       [0.00000000e+00, 1.19230438e-29, 2.38460875e-29, ...,
        1.18872746e-26, 1.18991976e-26, 1.19111207e-26],
       [0.00000000e+00, 1.19109754e-29, 2.38219508e-29, ...,
        1.18752424e-26, 1.18871534e-26, 1.18990643e-26]])